Which features appear to add the most value to a home?
pool

Which features hurt the value of a home the most?
fireplace

What are things that homeowners could improve in their homes to increase the value?
finished basement or screen porch

What neighborhoods seem like they might be a good investment?
NridgHT
StoneBr
NoRide


Do you feel that this model will generalize to other cities? How could you revise your model to make it more universal OR what date would you need from another city to make a comparable model?
No to generalize to other city we will need data from that city as no 2 cities are the same and we will data from the other cities to update our model

In [3]:
#Imports

import pandas as pd
import numpy as np

from sklearn.linear_model import Lasso, LassoCV, LinearRegression, Ridge, RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

pd.options.display.max_columns = 100
pd.options.display.max_columns = 3000

In [4]:
def merge_files():
    ames = pd.read_csv('../data/train.csv')
    ames_test = pd.read_csv('../data/test.csv')
    
    # Set test saleprice = 0    
    ames_test['saleprice'] = 0

    # Merge train and test
    final = pd.concat([ames, ames_test])

    return final

In [5]:
def sep_data(df):
    train = df[df['saleprice'] > 0]
    test = df[df['saleprice'] == 0]
    test.drop(['saleprice'], axis=1, inplace=True)
    return train, test

In [6]:
def evaluate_model(model, X, y):
    print(f'R2 Score: {r2_score(y, model.predict(X))}')
    print(f'MSE: {mean_squared_error(y, model.predict(X))}')
    print(f'RMSE: {np.sqrt(mean_squared_error(y, model.predict(X)))}')

In [7]:
def add_dummies(df, dummy_list):
    df = pd.get_dummies(columns=dummy_list, drop_first=True, data=df) #drop_first=True, drops the first column
    return df

In [9]:
def clean_data(df):
    df['lot_frontage'].fillna(0, inplace=True)
    df['alley'].fillna('NA', inplace=True)
    df['garage_area'].fillna(0, inplace=True)
    df['garage_cars'].fillna(0, inplace=True)
    df['total_bsmt_sf'].fillna(0, inplace=True)
    df['garage_yr_blt'].fillna(0, inplace=True)   
    df['mas_vnr_area'].fillna(0, inplace=True)
    df['bsmtfin_sf_1'].fillna(0, inplace=True)
    df['bsmtfin_sf_1'].fillna(0, inplace=True)
    df['electrical'].fillna('NA', inplace=True)
    df['bsmt_qual'].fillna('NA', inplace=True)
    df['fireplace_qu'].fillna('NA', inplace=True)
    df['mas_vnr_type'].fillna('NA', inplace=True)
    df['bsmt_cond'].fillna('NA', inplace=True)
    df['bsmt_exposure'].fillna('No', inplace=True)
    df['bsmtfin_type_1'].fillna('NA', inplace=True)
    df['bsmtfin_type_2'].fillna('NA', inplace=True)
    df['bsmtfin_sf_2'].fillna('NA', inplace=True)
    df['bsmt_unf_sf'].fillna(0, inplace=True)
    df['central_air'] = np.where(df['central_air'] == 'Y', 1, 0)
    df['bsmt_full_bath'].fillna(0, inplace=True)
    df['garage_finish'].fillna('NA', inplace=True)
    df['garage_cars'].fillna('NA', inplace=True)
    df['garage_qual'].fillna('NA', inplace=True)
    df['garage_cond'].fillna('NA', inplace=True)
    df['pool_qc'].fillna('NA', inplace=True)
    df['garage_type'].fillna(0, inplace=True)
    df['fence'].fillna('NA', inplace=True)
    df['misc_feature'].fillna('NA', inplace=True)
    df['bsmt_half_bath'].fillna(0, inplace=True)
    
    return df


In [10]:
# Model II - 
df = merge_files()

df = clean_data(df)

# Sales Price is included to be used to split the data into train and test dataset
all_data = df[['saleprice', "lot_area", "street", "lot_shape", "1st_flr_sf", "2nd_flr_sf", "yr_sold", "condition_1"
               , "neighborhood", "overall_cond", 'garage_area', 'year_remod/add', 'full_bath', 'totrms_abvgrd'
               , 'overall_qual', 'year_built', 'mas_vnr_area', 'bsmtfin_sf_1'
#                , 'ms_zoning', 'alley', 'land_contour', 'utilities', 'lot_config', 'roof_matl', 'exterior_1st'  #added to my model
#                , 'exter_cond', 'bsmt_qual', 'bsmt_exposure', 'bsmt_unf_sf', 'bedroom_abvgr', 'kitchen_qual'
#                , 'garage_qual', 'garage_cond'
#                , 'garage_qual', 'garage_cond', 'wood_deck_sf', 'enclosed_porch', '3ssn_porch'
#                , 'screen_porch', 'fence', 'misc_val', 'mo_sold', 'sale_type'
#                , 'land_slope', 'bldg_type', 'house_style', 'exterior_2nd', 'mas_vnr_type', 'exter_qual', 'bsmt_cond'
#                , 'bsmtfin_type_1' # not sure if this should stay
#                , 'bsmt_unf_sf', 'heating' , 'heating_qc', 'central_air', 'electrical', 'low_qual_fin_sf'
#                , 'bsmt_full_bath'
#                , 'bsmt_half_bath', 'half_bath', 'kitchen_abvgr', 'functional', 'fireplaces', 'fireplace_qu'
#                , 'garage_type'
#                , 'garage_finish', 'garage_cars', 'paved_drive', 'open_porch_sf', 'pool_area', 'pool_qc'
#                , 'misc_feature'
              ]]

dummy_list = ["street", "lot_shape", "yr_sold", "condition_1", "neighborhood", 'year_built'
#               , 'ms_zoning', 'alley', 'land_contour', 'utilities', 'lot_config', 'roof_matl', 'exterior_1st'
#               , 'exter_cond', 'bsmt_qual', 'bsmt_exposure', 'kitchen_qual', 'garage_qual', 'garage_cond'
#               , 'fence', 'mo_sold', 'sale_type'
#               , 'land_slope', 'bldg_type', 'house_style', 'exterior_2nd', 'mas_vnr_type', 'exter_qual', 'bsmtfin_type_1', 'bsmt_cond'
#               , 'heating', 'heating_qc', 'central_air', 'electrical', 'functional', 'fireplace_qu', 'garage_type'
#               , 'garage_finish', 'paved_drive', 'pool_qc', 'misc_feature'
             ]

all_data = add_dummies(all_data, dummy_list)

all_data

train, testset = sep_data(all_data)

X = train

y = train["saleprice"]
train.drop(['saleprice'], axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.01, random_state = 42)

lr = LinearRegression()
lr.fit(X_train, y_train)

print('+++++++ Train Results+++++++')
evaluate_model(lr, X_train, y_train)

print('+++++++ Test Results+++++++')
evaluate_model(lr, X_test, y_test)


test = pd.read_csv('../data/test.csv')
pred_price = lr.predict(testset)
test['y_hat'] = pred_price

submission = pd.DataFrame({'id': test['id'],
                            'SalePrice': test['y_hat']})

submission.to_csv('../data/prod_model.csv', index = False)

submission.head()

+++++++ Train Results+++++++
R2 Score: 0.9065165453468588
MSE: 589031241.1332144
RMSE: 24269.965824722836
+++++++ Test Results+++++++
R2 Score: 0.8905275429212176
MSE: 517214479.75781196
RMSE: 22742.34991723177


/Users/paulettewarrick/opt/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/paulettewarrick/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,id,SalePrice
0,2658,163210.659229
1,2718,171218.029074
2,2414,225600.525268
3,1989,110896.676559
4,625,172459.944726
